In [2]:
!pip3 install deepeval
!pip3 install pandas

  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.9/508.9 kB 6.4 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.9 MB/s eta 0:00:002 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00

In [7]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric, HallucinationMetric
from deepeval.test_case import LLMTestCase
import openai
import pandas as pd
import os

In [8]:
# Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] =  ""

In [17]:
# Load the data from the CSV file
df = pd.read_csv('RAG_evaluation.csv', encoding='latin1')
print(df.head())
# Extract the relevant columns
inputs = df['input'].tolist()
actual_outputs = df['actual_output'].tolist()
expected_outputs = df['expected_output'].tolist()
retrieval_context = df['retrieval_context'].apply(lambda x: [x] if isinstance(x, str) else None).tolist()

                                               input  \
0       What is the National Hydrogen Strategy 2050?   
1                    What is the Environmental Code?   
2  What are the targets of the Energy Policy Fram...   
3                       What is the Electricity Act?   
4  What is the Vision 2030 strategy for Saudi Ara...   

                                     expected_output  \
0  The National Hydrogen Strategy 2050 aims to su...   
1  The Environmental Code regulates activities im...   
2  The Energy Policy Framework aims to reduce ene...   
3  The Electricity Act sets regulations on energy...   
4  Saudi Arabia aims to increase the share of nat...   

                                   retrieval_context  \
0  ten points that the government will place effo...   
1  a number of policies affecting road traffic ha...   
2  the UK's 2nd Energy Efficiency Action Plan (EE...   
3  a number of policies affecting road traffic ha...   
4  ten points that the government will place e

In [18]:
# number of rows
n = len(df) 
print(n)

82


In [11]:
# Answer Relevancy Metric

# Initialize the metric
metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4o-mini",  # Ensure you have access to this model
    include_reason=True
)

# Create a list to hold test cases
test_cases = []

# Loop through the data to create test cases
for i in range(n):
    test_case = LLMTestCase(
        input=inputs[i],
        actual_output=actual_outputs[i]
    )
    test_cases.append(test_case)

# Evaluate test cases in bulk
evaluate(test_cases, [metric])

# Optionally, print the scores and reasons
for test_case in test_cases:
    if hasattr(test_case, 'metrics') and test_case.metrics:  # Ensure metrics are available
        print(f"Test case input: {test_case.input}")
        print(f"Answer Relevancy Score: {test_case.metrics[0].score}")
        print(f"Reason: {test_case.metrics[0].reason}")
    else:
        print(f"Metrics not available for input: {test_case.input}")
    print("="*50)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 82 test case(s) in parallel: |█████████████████████████████████████████████|100% (82/82) [Time Taken: 00:33,  2.44test case/s]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because the response directly addresses the question about the Environmental Code without including any irrelevant statements., error: None)

For test case:

  - input: What is the Environmental Code?
  - actual output: The Environmental Code is a legal framework designed to protect the outdoor environment against pollution and to reduce existing pollution levels. It establishes a general prohibition against pollution, requiring individuals and organizations to obtain specific permission before engaging in activities that may lead to environmental harm.
  - expected output: None
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because the response directly addresses the input regarding the National Climate C

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Metrics not available for input: What is the National Hydrogen Strategy 2050?
Metrics not available for input: What is the Environmental Code?
Metrics not available for input: What are the targets of the Energy Policy Framework (POPE)?
Metrics not available for input: What is the Electricity Act?
Metrics not available for input: What is the Vision 2030 strategy for Saudi ArabiaÂs energy sector?
Metrics not available for input: What is the purpose of the Decree of the Government on coal energy replacement?
Metrics not available for input: What is the phase-out of fossil fuels directive (2019:106)?
Metrics not available for input: What is the Mali's Nationally Determined Contribution (NDC)?
Metrics not available for input: What does the Presidential Instruction on Forest Moratorium entail?
Metrics not available for input: What is the Green Growth Strategy?
Metrics not available for input: What is the National Climate Change Plan aiming to reduce by 2030?
Metrics not available for input:

In [20]:
# Contextual Precision Metric
metric = ContextualPrecisionMetric(
    threshold=0.7,
    model="gpt-4o-mini",  # Ensure you have access to this model
    include_reason=True
)

# Create a list to hold test cases
test_cases = []

# Loop through the data to create 10 test cases
for i in range(n):
    test_case = LLMTestCase(
        input=inputs[i],
        actual_output=actual_outputs[i],
        expected_output=expected_outputs[i],
        retrieval_context=retrieval_context[i]
    )
    test_cases.append(test_case)

# Evaluate test cases in bulk
evaluate(test_cases, [metric])

# Optionally, print the scores and reasons
for test_case in test_cases:
    if hasattr(test_case, 'metrics') and test_case.metrics:  # Ensure metrics are available
        print(f"Test case input: {test_case.input}")
        print(f"Score: {test_case.metrics[0].score}")
        print(f"Reason: {test_case.metrics[0].reason}")
    else:
        print(f"Metrics not available for input: {test_case.input}")
    print("="*50)

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 82 test case(s) in parallel: |█████████████████████████████████████████████|100% (82/82) [Time Taken: 00:24,  3.35test case/s]



Metrics Summary

  - ❌ Contextual Precision (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.00 because the first node is irrelevant, stating that it 'primarily discusses Norway's strategy for the mineral industry,' which does not relate to the Green Growth Strategy, resulting in no relevant nodes being ranked higher., error: None)

For test case:

  - input: What is the Green Growth Strategy?
  - actual output: The Green Growth Strategy is a comprehensive plan aimed at establishing Norway as the most sustainable mineral industry in the world. This strategy is part of a broader initiative to promote green finance and sustainable practices within the country. It outlines specific goals and policies to reduce emissions across key sectors, including industry, transport, and buildings, while also focusing on clean power generation.

Key components of the strategy include:
1. Achieving 40 GW of offshore wind power production by 2030.
2. Cap

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Metrics not available for input: What is the National Hydrogen Strategy 2050?
Metrics not available for input: What is the Environmental Code?
Metrics not available for input: What are the targets of the Energy Policy Framework (POPE)?
Metrics not available for input: What is the Electricity Act?
Metrics not available for input: What is the Vision 2030 strategy for Saudi ArabiaÂs energy sector?
Metrics not available for input: What is the purpose of the Decree of the Government on coal energy replacement?
Metrics not available for input: What is the phase-out of fossil fuels directive (2019:106)?
Metrics not available for input: What is the Mali's Nationally Determined Contribution (NDC)?
Metrics not available for input: What does the Presidential Instruction on Forest Moratorium entail?
Metrics not available for input: What is the Green Growth Strategy?
Metrics not available for input: What is the National Climate Change Plan aiming to reduce by 2030?
Metrics not available for input:

In [21]:
# Contextual Recall Metric
metric = ContextualRecallMetric(
    threshold=0.7,
    model="gpt-4o-mini",  # Ensure you have access to this model
    include_reason=True
)

# Create a list to hold test cases
test_cases = []

# Loop through the data to create test cases
for i in range(n):
    test_case = LLMTestCase(
        input=inputs[i],
        actual_output=actual_outputs[i],
        expected_output=expected_outputs[i],
        retrieval_context=retrieval_context[i]
    )
    test_cases.append(test_case)

# Evaluate test cases in bulk
evaluate(test_cases, [metric])

# Optionally, print the scores and reasons
for test_case in test_cases:
    if hasattr(test_case, 'metrics') and test_case.metrics:  # Ensure metrics are available
        print(f"Test case input: {test_case.input}")
        print(f"Recall Score: {test_case.metrics[0].score}")
        print(f"Reason: {test_case.metrics[0].reason}")
    else:
        print(f"Metrics not available for input: {test_case.input}")
    print("="*50)

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 82 test case(s) in parallel: |█████████████████████████████████████████████|100% (82/82) [Time Taken: 00:22,  3.68test case/s]



Metrics Summary

  - ❌ Contextual Recall (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The score is 0.00 because the content of the expected output, particularly regarding the National Hydrogen Strategy 2050 and its focus on 10 enablers, is not addressed in any of the nodes in the retrieval context., error: None)

For test case:

  - input: What is the National Hydrogen Strategy 2050?
  - actual output: The National Hydrogen Strategy 2050 is a long-term framework developed by the government to guide the transition towards a sustainable and low-carbon energy system, with a particular focus on hydrogen as a key energy source. The strategy outlines several key priorities and actions aimed at achieving significant milestones in the next few decades, particularly by 2030 and beyond. 

Key points of the strategy include:

1. **Clean Power Generation**: Enhancing the production of clean energy, with a target of generating 40 GW of offshore wind power by 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Metrics not available for input: What is the National Hydrogen Strategy 2050?
Metrics not available for input: What is the Environmental Code?
Metrics not available for input: What are the targets of the Energy Policy Framework (POPE)?
Metrics not available for input: What is the Electricity Act?
Metrics not available for input: What is the Vision 2030 strategy for Saudi ArabiaÂs energy sector?
Metrics not available for input: What is the purpose of the Decree of the Government on coal energy replacement?
Metrics not available for input: What is the phase-out of fossil fuels directive (2019:106)?
Metrics not available for input: What is the Mali's Nationally Determined Contribution (NDC)?
Metrics not available for input: What does the Presidential Instruction on Forest Moratorium entail?
Metrics not available for input: What is the Green Growth Strategy?
Metrics not available for input: What is the National Climate Change Plan aiming to reduce by 2030?
Metrics not available for input:

In [22]:
# FaithfulnessMetric
metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-4o-mini",  # Ensure you have access to this model
    include_reason=True
)

# Create a list to hold test cases
test_cases = []

# Loop through the data to create test cases
for i in range(n):
    test_case = LLMTestCase(
        input=inputs[i],
        actual_output=actual_outputs[i],
        retrieval_context=retrieval_context[i]
    )
    test_cases.append(test_case)

# Evaluate test cases in bulk
evaluate(test_cases, [metric])

# Optionally, print the scores and reasons
for test_case in test_cases:
    if hasattr(test_case, 'metrics') and test_case.metrics:  # Ensure metrics are available
        print(f"Test case input: {test_case.input}")
        print(f"Faithfulness Score: {test_case.metrics[0].score}")
        print(f"Reason: {test_case.metrics[0].reason}")
    else:
        print(f"Metrics not available for input: {test_case.input}")
    print("="*50)


✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 82 test case(s) in parallel: |█████████████████████████████████████████████|100% (82/82) [Time Taken: 00:42,  1.92test case/s]



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The score is 1.00 because there are no contradictions, indicating that the actual output perfectly aligns with the retrieval context., error: None)

For test case:

  - input: Can you explain what carbon footprint means?
  - actual output: A carbon footprint refers to the total amount of greenhouse gases, primarily carbon dioxide (CO2), that are emitted directly or indirectly by an individual, organization, event, or product throughout its lifecycle. It is usually measured in equivalent tons of CO2 and takes into account all activities that contribute to these emissions, such as energy consumption, transportation, agricultural practices, and production processes.

In the context of the projects mentioned in Norway, the focus on quantifying greenhouse gas emissions from farms is essential for understanding and reducing their carbon footprints. By developing tools to m

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Metrics not available for input: What is the National Hydrogen Strategy 2050?
Metrics not available for input: What is the Environmental Code?
Metrics not available for input: What are the targets of the Energy Policy Framework (POPE)?
Metrics not available for input: What is the Electricity Act?
Metrics not available for input: What is the Vision 2030 strategy for Saudi ArabiaÂs energy sector?
Metrics not available for input: What is the purpose of the Decree of the Government on coal energy replacement?
Metrics not available for input: What is the phase-out of fossil fuels directive (2019:106)?
Metrics not available for input: What is the Mali's Nationally Determined Contribution (NDC)?
Metrics not available for input: What does the Presidential Instruction on Forest Moratorium entail?
Metrics not available for input: What is the Green Growth Strategy?
Metrics not available for input: What is the National Climate Change Plan aiming to reduce by 2030?
Metrics not available for input:

In [24]:
# Hallucination Metric
metric = HallucinationMetric(threshold=0.5)

# Create a list to hold test cases
test_cases = []

# Loop through the data to create test cases
for i in range(n):
    test_case = LLMTestCase(
        input=inputs[i],
        actual_output=actual_outputs[i],
        context=retrieval_context[i]
    )
    test_cases.append(test_case)

# Evaluate test cases in bulk
evaluate(test_cases, [metric])

# Optionally, print the scores and reasons
for test_case in test_cases:
    if hasattr(test_case, 'metrics') and test_case.metrics:  # Ensure metrics are available
        print(f"Test case input: {test_case.input}")
        print(f"Hallucination Score: {test_case.metrics[0].score}")
        print(f"Reason: {test_case.metrics[0].reason}")
    else:
        print(f"Metrics not available for input: {test_case.input}")
    print("="*50)

✨ You're running DeepEval's latest Hallucination Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 82 test case(s) in parallel: |█████████████████████████████████████████████|100% (82/82) [Time Taken: 00:23,  3.53test case/s]



Metrics Summary

  - ✅ Hallucination (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the actual output fully aligns with the provided context, with no contradictions present., error: None)

For test case:

  - input: What is the Environmental Code?
  - actual output: The Environmental Code is a legal framework designed to protect the outdoor environment against pollution and to reduce existing pollution levels. It establishes a general prohibition against pollution, requiring individuals and organizations to obtain specific permission before engaging in activities that may lead to environmental harm.
  - expected output: None
  - context: ['a number of policies affecting road traffic have been implemented in Norway. a number of policies have been implemented in the country. a number of policies have been implemented in the country. the minimum harvesting age is the minimum harvesting age. the minimum harvesting age is the minimu

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Metrics not available for input: What is the National Hydrogen Strategy 2050?
Metrics not available for input: What is the Environmental Code?
Metrics not available for input: What are the targets of the Energy Policy Framework (POPE)?
Metrics not available for input: What is the Electricity Act?
Metrics not available for input: What is the Vision 2030 strategy for Saudi ArabiaÂs energy sector?
Metrics not available for input: What is the purpose of the Decree of the Government on coal energy replacement?
Metrics not available for input: What is the phase-out of fossil fuels directive (2019:106)?
Metrics not available for input: What is the Mali's Nationally Determined Contribution (NDC)?
Metrics not available for input: What does the Presidential Instruction on Forest Moratorium entail?
Metrics not available for input: What is the Green Growth Strategy?
Metrics not available for input: What is the National Climate Change Plan aiming to reduce by 2030?
Metrics not available for input: